In [1]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize

In [7]:
state_info = pd.read_csv('stats_all.csv')

In [8]:
state_info

,state,city,total,total_correct_city,max_dist,max_dist_city,bad_dates
0,AL,Birmingham,39005,1445.0,99.6295,13.3825,714.0
1,AK,Anchorage,2769,2516.0,78.5158,8.8004,102.0
2,AZ,Phoenix,59766,17393.0,99.8284,32.4871,1154.0
3,AR,Little Rock,22584,509.0,97.0427,10.2831,184.0
4,CA,Los Angeles,44687,28564.0,10.4834,10.4192,301.0
5,CO,Denver,85222,8587.0,99.8258,10.8111,1684.0
6,CT,Bridgeport,11291,1036.0,15.9824,2.3724,406.0
7,DE,Wilmington,-1,NaN,NaN,NaN,NaN
8,FL,Jacksonville,-1,NaN,NaN,NaN,NaN
9,GA,Atlanta,-1,NaN,NaN,NaN,NaN


In [10]:
from datetime import datetime
def get_diff(end, start):
    end_d = datetime.strptime(end,"%Y-%m-%dT%H:%M:%S%z")
    start_d = datetime.strptime(start,"%Y-%m-%dT%H:%M:%S%z")
    if(end_d == start_d):
        return -1
    return (end_d - start_d).days

In [11]:
#from datetime import datetime
def get_month(date):
    z = datetime.strptime(date,"%Y-%m-%dT%H:%M:%S%z")
    return z.month

In [12]:
#from datetime import datetime
def get_year(date):
    z = datetime.strptime(date,"%Y-%m-%dT%H:%M:%S%z")
    return z.year

In [17]:
for i in range(0,49):
    current_state = state_info['state'][i]
    current_city = state_info['city'][i]
    print('Processing state {} - {}'.format(i, current_state))
    alldogs = pd.read_csv('./data_all/alldogs_{}.csv'.format(current_state))
    alldogs_reduced = alldogs.drop(['type', 'species', 'description', 'photos', 
                                    'contact.email', 'contact.phone', 'contact.address.address1', 
                                    'contact.address.address2', 'contact.address.country', 
                                    '_links.self.href', '_links.type.href', '_links.organization.href'], axis=1)
    alldogs_reduced = alldogs_reduced.drop_duplicates()
    alldogs_reduced = alldogs_reduced[alldogs_reduced['contact.address.state']==current_state]
    alldogs_reduced['adoption_time'] = alldogs_reduced.apply(lambda x: get_diff(x['status_changed_at'], x['published_at']), axis=1)
    bad_dates = len(alldogs_reduced[alldogs_reduced['adoption_time']==-1])
    alldogs_reduced = alldogs_reduced[alldogs_reduced['adoption_time']!=-1]
    alldogs_reduced['year'] = alldogs_reduced.apply(lambda x: get_year(x['status_changed_at']), axis=1)
    alldogs_reduced['month'] = alldogs_reduced.apply(lambda x: get_month(x['status_changed_at']), axis=1)
    state_info.loc[i , 'max_dist'] = alldogs_reduced['distance'].max()
    state_info.loc[i , 'bad_dates'] = bad_dates
    export = state_info.to_csv('stats_all.csv', header=True, index=False)
    export = alldogs_reduced.to_csv('./data_all/alldogs_reduced_{}.csv'.format(current_state), header=True, index = False)

Processing state 0 - AL
Processing state 1 - AK
Processing state 2 - AZ
Processing state 3 - AR
Processing state 4 - CA
Processing state 5 - CO
Processing state 6 - CT
Processing state 7 - DE
Processing state 8 - FL
Processing state 9 - GA


C:\anaconda_python\envs\Insight\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Processing state 10 - ID
Processing state 11 - IL


C:\anaconda_python\envs\Insight\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Processing state 12 - IN
Processing state 13 - IA
Processing state 14 - KS
Processing state 15 - KY
Processing state 16 - LA
Processing state 17 - ME
Processing state 18 - MD
Processing state 19 - MA
Processing state 20 - MI


C:\anaconda_python\envs\Insight\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Processing state 21 - MN
Processing state 22 - MS
Processing state 23 - MO
Processing state 24 - MT
Processing state 25 - NE
Processing state 26 - NV
Processing state 27 - NH
Processing state 28 - NJ
Processing state 29 - NM
Processing state 30 - NY
Processing state 31 - NC
Processing state 32 - ND
Processing state 33 - OH
Processing state 34 - OK
Processing state 35 - OR
Processing state 36 - PA
Processing state 37 - RI
Processing state 38 - SC
Processing state 39 - SD
Processing state 40 - TN
Processing state 41 - TX
Processing state 42 - UT
Processing state 43 - VT
Processing state 44 - VA
Processing state 45 - WA
Processing state 46 - WV
Processing state 47 - WI
Processing state 48 - WY


In [4]:
alldogs = pd.read_csv('./data/alldogs_{}.csv'.format(current_state))

C:\anaconda_python\envs\Insight\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
readfile

'alldogs_AL'

In [5]:
alldogs

,id,organization_id,url,type,species,age,gender,size,coat,tags,...,contact.phone,contact.address.address1,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,_links.self.href,_links.type.href,_links.organization.href
0,2604949,NY114,https://www.petfinder.com/dog/lexie-2604949/ny...,Dog,Dog,Young,Female,Large,NaN,[],...,(212) 252-3183,P.O. Box 1345,Canal Street Station,New York,NY,10013,US,/v2/animals/2604949,/v2/types/dog,/v2/organizations/ny114
1,3821543,NY114,https://www.petfinder.com/dog/poipu-3821543/ny...,Dog,Dog,Adult,Female,Small,NaN,[],...,(212) 252-3183,P.O. Box 1345,Canal Street Station,New York,NY,10013,US,/v2/animals/3821543,/v2/types/dog,/v2/organizations/ny114
2,3924002,NY114,https://www.petfinder.com/dog/nellie-3924002/n...,Dog,Dog,Adult,Female,Small,NaN,[],...,(212) 252-3183,P.O. Box 1345,Canal Street Station,New York,NY,10013,US,/v2/animals/3924002,/v2/types/dog,/v2/organizations/ny114
3,4115299,NY114,https://www.petfinder.com/dog/kiko-4115299/ny/...,Dog,Dog,Adult,Male,Small,NaN,[],...,(212) 252-3183,P.O. Box 1345,Canal Street Station,New York,NY,10013,US,/v2/animals/4115299,/v2/types/dog,/v2/organizations/ny114
4,4127857,NY114,https://www.petfinder.com/dog/princess-4127857...,Dog,Dog,Senior,Female,Small,NaN,[],...,212-691-2053,P.O. Box 1345,Canal Street Station,New York,NY,10013,US,/v2/animals/4127857,/v2/types/dog,/v2/organizations/ny114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,37886284,VA524,https://www.petfinder.com/dog/daisy-37886284/n...,Dog,Dog,Baby,Female,Small,Short,[],...,Please Email Us!,NaN,NaN,Madison,NJ,7940,US,/v2/animals/37886284,/v2/types/dog,/v2/organizations/va524
89996,37919613,VA524,https://www.petfinder.com/dog/willie-37919613/...,Dog,Dog,Young,Male,Small,Medium,[],...,Please Email Us!,NaN,NaN,Madison,NJ,7940,US,/v2/animals/37919613,/v2/types/dog,/v2/organizations/va524
89997,37926806,VA524,https://www.petfinder.com/dog/amber-37926806/n...,Dog,Dog,Young,Female,Medium,Short,[],...,Please Email Us!,NaN,NaN,Madison,NJ,7940,US,/v2/animals/37926806,/v2/types/dog,/v2/organizations/va524
89998,37935417,VA524,https://www.petfinder.com/dog/ensa-37935417/nj...,Dog,Dog,Baby,Female,Small,Short,[],...,Please Email Us!,NaN,NaN,Madison,NJ,7940,US,/v2/animals/37935417,/v2/types/dog,/v2/organizations/va524
